# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

import pickle

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
def load_data(db_file_name,engine):
    """
    Load data from database
    
    Arguments:
        db_file_name ->  dataset cleaned by ETL pipeline 
        engine       -> database
    Output:
        X and Y data
    
    """
    df     = pd.read_sql_table(db_file_name, engine)
    X      = df['message']
    Y      = df.iloc[:,4:]
    return ( X,Y )
####
# load data from database
db_file_name = 'disaster_response_data'
engine       = create_engine('sqlite:///disaster_response_data.db')
X,Y          = load_data(db_file_name,engine)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
####

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [5]:
def build_model():
    """
    Builds a machine learning model 
    Arguments:
        None
    Output:
        model
    """
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf',MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline
####

In [5]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf',MultiOutputClassifier(AdaBoostClassifier()))
    ])
    parameters = {'clf__estimator__n_estimators': [10, 20, 40]}
    cv = GridSearchCV(pipeline, param_grid=parameters)
    return (cv)
####

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

#Training Pipeline
model = build_model().fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_test_pred         =  model.predict(X_test)
Y_test_pred_df      =  pd.DataFrame(Y_test_pred , columns = Y_test.columns)

In [ ]:
for each_col in Y_test.columns : 
    print ("\n     >========================================<     \nCategory : %s"%each_col)
    print ( classification_report(Y_test[each_col] , Y_test_pred_df[each_col]) )
###

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_p

In [11]:
def Improve_model(pipeline,parameters) : 
    cv          = GridSearchCV(pipeline, param_grid=parameters)
    return(cv)
####

parameters     = {'clf__estimator__n_estimators': [10, 20, 40]}
pipeline       = build_model()
model_improved = Improve_model(pipeline,parameters)



In [12]:
model_improved.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
Y_pred_test = model_improved.predict(X_test)

In [14]:
Y_pred_test_df      =  pd.DataFrame(Y_pred_test , columns = Y_test.columns)
for each_col in Y_test.columns : 
    print ("\n     >========================================<     \nCategory : %s"%each_col)
    print ( classification_report(Y_test[each_col] , Y_test_pred_df[each_col]) )
###


     >========================================<     
Category : related
             precision    recall  f1-score   support

          0       0.61      0.24      0.34      1592
          1       0.79      0.95      0.86      4914
          2       0.45      0.19      0.26        48

avg / total       0.74      0.77      0.73      6554


     >========================================<     
Category : request
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5451
          1       0.75      0.57      0.65      1103

avg / total       0.89      0.90      0.89      6554


     >========================================<     
Category : offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      6554


     >========================================<     
Category : aid_related
        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
Note : Already tried two different classifiers. 

### 9. Export your model as a pickle file

In [17]:

pickle.dump(model_improved, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.